**What we did in this notebook:**

Implemented content filtering recommendation system using Cosine distances


*Implementing our model:*

We load data that we saved as csv from our 'final_project_data_cleaning.ipynb'

In order to recommend songs we use Cosine distances of songs' audio features that we get from Spotify API with other songs' features.

The theory is that if songs share similar audio features (a short cosine distance), they are similar songs.

Cosine similarity measures the orientation of two n-dimensional sample vectors irrespective to their magnitude. It is calculated by the dot product of two numeric vectors, and it is normalized by the product of the vector lengths. The output is in the range 0 to 1, with 1 being the highest similarity.

We will be using cosine_similarity function from sklearn.pairwise to implement our model.
We created 2 generators that are used in Spotify:
1. Radio generator (Generates a playlist based on 1 song)
2. Playlist generator (Generates n playlists based on 1 playlist)


In [66]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json

from sklearn.model_selection import cross_val_score
from sklearn.utils import resample
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity 
from sklearn.preprocessing import MinMaxScaler

%matplotlib inline

import seaborn as sns
sns.set(style='whitegrid')
pd.set_option('display.width', 1500)
pd.set_option('display.max_columns', 100)

In [112]:
# Reading the dataframe to start working
df = pd.read_csv("data/Sample_from_Million_Playlist.csv")

In [113]:
# Reading the data from saved file
# This is the dataframe we will implement our mode
df_spotify = pd.read_csv("data/100_Sample_MilPlay_Spotify.csv")
df_spotify.head(10)

,pid,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms.1,time_signature
0,4,0,Alessia Cara,spotify:track:1wYZZtamWTQAoj8B812uKQ,spotify:artist:2wUjUUtkb5lvLKcGKsKqsR,Here,spotify:album:3rDbA12I5duZnlwakqDdZa,199453,Know-It-All,0.376,0.821,0,-3.974,1,0.1040,0.0785,0.000000,0.0823,0.331,120.462,audio_features,1wYZZtamWTQAoj8B812uKQ,spotify:track:1wYZZtamWTQAoj8B812uKQ,https://api.spotify.com/v1/tracks/1wYZZtamWTQA...,https://api.spotify.com/v1/audio-analysis/1wYZ...,199453,4
1,4,1,The Weeknd,spotify:track:0Gi17qCJh9e9RJxLaYkm9l,spotify:artist:1Xyo4u8uXC1ZmMpatF05PJ,Dark Times,spotify:album:28ZKQMoNBB0etKXZ97G2SN,260640,Beauty Behind The Madness,0.585,0.421,7,-9.593,1,0.0707,0.1060,0.000010,0.1400,0.240,132.986,audio_features,0Gi17qCJh9e9RJxLaYkm9l,spotify:track:0Gi17qCJh9e9RJxLaYkm9l,https://api.spotify.com/v1/tracks/0Gi17qCJh9e9...,https://api.spotify.com/v1/audio-analysis/0Gi1...,260640,3
2,4,2,J. Cole,spotify:track:6Ius4TC0L3cN74HT7ENE6e,spotify:artist:6l3HvQ5sa6mXTsMTB19rO5,Wet Dreamz,spotify:album:7viNUmZZ8ztn2UB4XB3jIL,239320,2014 Forest Hills Drive,0.504,0.705,6,-8.205,0,0.3640,0.0752,0.000000,0.1280,0.584,175.483,audio_features,6Ius4TC0L3cN74HT7ENE6e,spotify:track:6Ius4TC0L3cN74HT7ENE6e,https://api.spotify.com/v1/tracks/6Ius4TC0L3cN...,https://api.spotify.com/v1/audio-analysis/6Ius...,239320,4
3,4,3,Chance The Rapper,spotify:track:0jx8zY5JQsS4YEQcfkoc5C,spotify:artist:1anyVhU62p31KFi8MEzkbf,Angels (feat. Saba),spotify:album:71QyofYesSsRMwFOTafnhB,206240,Coloring Book,0.771,0.647,5,-5.127,0,0.3760,0.2940,0.000000,0.3700,0.678,155.914,audio_features,0jx8zY5JQsS4YEQcfkoc5C,spotify:track:0jx8zY5JQsS4YEQcfkoc5C,https://api.spotify.com/v1/tracks/0jx8zY5JQsS4...,https://api.spotify.com/v1/audio-analysis/0jx8...,206240,4
4,4,4,The Weeknd,spotify:track:7fPHfBCyKE3aVCBjE4DAvl,spotify:artist:1Xyo4u8uXC1ZmMpatF05PJ,In The Night,spotify:album:28ZKQMoNBB0etKXZ97G2SN,235653,Beauty Behind The Madness,0.480,0.682,7,-4.940,1,0.1300,0.0696,0.000000,0.0463,0.506,167.939,audio_features,7fPHfBCyKE3aVCBjE4DAvl,spotify:track:7fPHfBCyKE3aVCBjE4DAvl,https://api.spotify.com/v1/tracks/7fPHfBCyKE3a...,https://api.spotify.com/v1/audio-analysis/7fPH...,235653,3
5,4,5,Donnie Trumpet & The Social Experiment,spotify:track:6fTdcGsjxlAD9PSkoPaLMX,spotify:artist:0ojcq9LJQWMawQdFDw3M0L,Sunday Candy,spotify:album:3eM1KTKmpqrQOvuvYY42cr,226013,Surf,0.511,0.596,0,-6.560,1,0.2240,0.5300,0.000000,0.0798,0.554,158.063,audio_features,6fTdcGsjxlAD9PSkoPaLMX,spotify:track:6fTdcGsjxlAD9PSkoPaLMX,https://api.spotify.com/v1/tracks/6fTdcGsjxlAD...,https://api.spotify.com/v1/audio-analysis/6fTd...,226014,4
6,4,6,Beyoncé,spotify:track:2CvOqDpQIMw69cCzWqr5yr,spotify:artist:6vWDO969PvNqNYHIOW5v0m,Halo,spotify:album:3ROfBX6lJLnCmaw1NrP5K9,261160,I AM...SASHA FIERCE - Platinum Edition,0.422,0.712,11,-5.907,0,0.1000,0.2730,0.000000,0.0510,0.471,78.454,audio_features,2CvOqDpQIMw69cCzWqr5yr,spotify:track:2CvOqDpQIMw69cCzWqr5yr,https://api.spotify.com/v1/tracks/2CvOqDpQIMw6...,https://api.spotify.com/v1/audio-analysis/2CvO...,261160,4
7,4,7,Hozier,spotify:track:1ivHxaGL5ld9VS1zsYc4YN,spotify:artist:2FXC3k01G6Gw61bmprjgqS,Cherry Wine - Live,spotify:album:36k5aXpxffjVGcNce12GLZ,240147,Hozier,0.418,0.111,1,-14.848,1,0.0389,0.9530,0.003420,0.0982,0.228,82.508,audio_features,1ivHxaGL5ld9VS1zsYc4YN,spotify:track:1ivHxaGL5ld9VS1zsYc4YN,https://api.spotify.com/v1/tracks/1ivHxaGL5ld9...,https://api.spotify.com/v1/audio-analysis/1ivH...,240147,4
8,4,8,Hozier,spotify:track:1TGimSbipZ3XZ7q3eszBRV,spotify:artist:2FXC3k01G6Gw61bmprjgqS,Angel Of Small Death & The Codeine Scene,spotify:album:36k5aXpxffjVGcNce12GLZ,219214,Hozier,0.377,0.638,4,-5.754,1,0.0545,0.2130,0.000080,0.1200,0.369,92.644,audio_features,1TGimSbipZ3XZ7q3eszBRV,spotify:track:1TGimSbipZ3XZ7q3eszBRV,https://api.spotify.com/v1/tracks/1TGimSbipZ3X...,ht

In [114]:
# Cleaning the data frame and removing features we don't need
df_spotify = df_spotify.drop_duplicates(['track_uri']).reset_index()
df_spotify_tracks = df_spotify['track_uri']
df_spotify_track_names = df_spotify['track_name']

# We will be using the clean df to implement our model
df_spotify_clean = df_spotify.drop(columns=['index', 'pid','pos','artist_name','track_uri','artist_uri','track_name','album_uri','album_name','type','id','uri','track_href','analysis_url','duration_ms'])
df_spotify_clean.head(10)

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms.1,time_signature
0,0.376,0.821,0,-3.974,1,0.1040,0.0785,0.000000,0.0823,0.331,120.462,199453,4
1,0.585,0.421,7,-9.593,1,0.0707,0.1060,0.000010,0.1400,0.240,132.986,260640,3
2,0.504,0.705,6,-8.205,0,0.3640,0.0752,0.000000,0.1280,0.584,175.483,239320,4
3,0.771,0.647,5,-5.127,0,0.3760,0.2940,0.000000,0.3700,0.678,155.914,206240,4
4,0.480,0.682,7,-4.940,1,0.1300,0.0696,0.000000,0.0463,0.506,167.939,235653,3
5,0.511,0.596,0,-6.560,1,0.2240,0.5300,0.000000,0.0798,0.554,158.063,226014,4
6,0.422,0.712,11,-5.907,0,0.1000,0.2730,0.000000,0.0510,0.471,78.454,261160,4
7,0.418,0.111,1,-14.848,1,0.0389,0.9530,0.003420,0.0982,0.228,82.508,240147,4
8,0.377,0.638,4,-5.754,1,0.0545,0.2130,0.000080,0.1200,0.369,92.644,219214,4
9,0.395,0.676,0,-5.460,1,0.0498,0.6080,0.000049,0.1170,0.315,142.929,283467,5


In [109]:
# Standardizing the data
scaler = MinMaxScaler()
scaler.fit(df_spotify_clean)
df_spotify_clean_scaled = scaler.transform(df_spotify_clean)

# We create the cosine similarity matrix of the small scaled dataframe we have 
df_spotify_cosine = cosine_similarity(df_spotify_clean_scaled)


In [78]:
# We save the cosine similarity arrays as dataframes such that each column gives the similarity one song has with every other song
df_spotify_cosine = pd.DataFrame(df_spotify_cosine)
df_spotify_cosine.head(10)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,...,9341,9342,9343,9344,9345,9346,9347,9348,9349,9350,9351,9352,9353,9354,9355,9356,9357,9358,9359,9360,9361,9362,9363,9364,9365,9366,9367,9368,9369,9370,9371,9372,9373,9374,9375,9376,9377,9378,9379,9380,9381,9382,9383,9384,9385,9386,9387,9388,9389,9390
0,1.000000,0.901929,0.782104,0.768605,0.929427,0.952791,0.704712,0.798435,0.971115,0.956404,0.804617,0.656117,0.924023,0.712121,0.895556,0.709808,0.816971,0.838186,0.759852,0.771055,0.848531,0.933460,0.894670,0.851069,0.910214,0.751266,0.798300,0.916668,0.776012,0.854353,0.981741,0.878649,0.852297,0.967445,0.807738,0.830235,0.813581,0.893107,0.856532,0.751023,0.780303,0.826329,0.672799,0.881161,0.758602,0.776846,0.975990,0.822804,0.865163,0.839222,...,0.713278,0.869824,0.896826,0.709012,0.825490,0.745285,0.988602,0.932940,0.944161,0.802150,0.939805,0.910416,0.985648,0.806747,0.748022,0.767741,0.986608,0.981791,0.990721,0.935329,0.991845,0.924825,0.990590,0.883254,0.974323,0.954247,0.772410,0.929324,0.991472,0.989693,0.924388,0.735684,0.928009,0.730468,0.794967,0.722420,0.882501,0.928154,0.925783,0.806396,0.934373,0.981311,0.805707,0.954403,0.983465,0.727270,0.800813,0.853179,0.726925,0.822481
1,0.901929,1.000000,0.793016,0.784660,0.977052,0.895163,0.787653,0.810602,0.958200,0.883065,0.934328,0.808315,0.987625,0.771784,0.899791,0.869206,0.876955,0.856533,0.732228,0.700819,0.953584,0.983008,0.979680,0.883688,0.990670,0.822082,0.800197,0.968612,0.782559,0.981413,0.902625,0.958088,0.968871,0.922734,0.775614,0.905028,0.808688,0.952769,0.828141,0.816685,0.689450,0.784084,0.763690,0.958329,0.705311,0.721822,0.966398,0.768118,0.970312,0.837960,...,0.737182,0.857650,0.811276,0.739321,0.764768,0.727575,0.925147,0.934282,0.942909,0.734376,0.957503,0.959847,0.888393,0.769362,0.792096,0.768625,0.891964,0.894465,0.908489,0.923030,0.920633,0.939517,0.894091,0.947318,0.867078,0.937729,0.687527,0.933926,0.908407,0.904241,0.959847,0.782222,0.963220,0.782390,0.731275,0.792868,0.956223,0.961318,0.928315,0.692460,0.945796,0.899743,0.788914,0.882595,0.895465,0.785291,0.778550,0.912718,0.634707,0.767966
2,0.782104,0.793016,1.000000,0.973037,0.851803,0.798792,0.918544,0.586112,0.797468,0.763012,0.759194,0.890711,0.827118,0.861079,0.742305,0.643266,0.722593,0.687334,0.860089,0.842453,0.795338,0.812014,0.833190,0.764368,0.828459,0.935405,0.952947,0.846000,0.957549,0.834544,0.792923,0.745959,0.820177,0.745273,0.954160,0.693827,0.594258,0.715039,0.656240,0.935515,0.893171,0.956374,0.877879,0.822125,0.840717,0.897295,0.822096,0.937097,0.774265,0.636774,...,0.884961,0.767591,0.679021,0.876631,0.942432,0.889531,0.823967,0.813327,0.817252,0.907036,0.832438,0.857655,0.790348,0.969155,0.934746,0.931267,0.792509,0.807401,0.784257,0.813844,0.824072,0.837256,0.792045,0.810760,0.742562,0.797463,0.851686,0.810332,0.812263,0.785111,0.823543,0.920936,0.833663,0.905589,0.916684,0.914852,0.834711,0.834362,0.803617,0.875673,0.819295,0.759280,0.958868,0.762531,0.796715,0.917604,0.922885,0.764699,0.774391,0.929519
3,0.768605,0.784660,0.973037,1.000000,0.822618,0.819227,0.894000,0.647500,0.791634,0.777361,0.762265,0.891221,0.811168,0.887975,0.773242,0.680460,0.767974,0.756680,0.899000,0.901862,0.811657,0.816071,0.821090,0.818471,0.815553,0.935979,0.946546,0.812633,0.954119,0.831972,0.811696,0.735179,0.811987,0.765635,0.952866,0.681707,0.639078,0.715040,0.718020,0.948820,0.928096,0.958370,0.902304,0.831195,0.898883,0.927927,0.800233,0.952067,0.764260,0.705855,...,0.833104,0.730343,0.705829,0.818188,0.905117,0.891443,0.803364,0.797466,0.780061,0.885374,0.814528,0.813439,0.778076,0.953829,0.911675,0.930184,0.792822,0.809080,0.772799,0.813936,0.798709,0.822443,0.776007,0.764325,0.720907,0.775025,0.841921,0.795193,0.791291,0.770289,0.799148,0.882436,0.795576,0.878228,0.887471,0.890335,0.799061,0.811843,0.786815,0.876713,0.797486,0.731633,0.933108,0.764892,0.796239,0.882490,0.899056,0.710437

**Recommendation Approach:**

Let's create two recommendation function.
We will be use a content filtering method to recommend songs.
We will look at features of songs found from Spotify API and then use Cosine similarity to provide the recommendations.

In [79]:

# This is the radio generator, where you give a song and Spotify suggests you X number of songs based on one song

# This functions assume that the song you provide is already in the list of 9,300 songs we have for
# randomly selected 100 playlists

def generate_radio(uri, cosine_df, info_df, num_tracks = 10):
    '''
    Input:
    uri = Track to provide recommendations on
    cosine_df = Cosine similarity df
    info_df = The df with all the songs and track info and audio features
    num_tracks = Number of tracks to recommend
    
    Output:
    rec_songs = df of track names and uri of recommended songs, length = num_tracks 
    '''
    index = info_df.index[info_df['track_uri'] == uri][0]
    similarities = cosine_df.iloc[:, index].sort_values(ascending=False)
    final_indices = list(similarities[1:num_tracks].index)
    rec_songs = info_df[['track_name','artist_name','track_uri']].iloc[final_indices]
    return rec_songs

In [80]:
# Testing our radio generating function

i = 34
test_uri = df_spotify_tracks[i]
test_pid = df_spotify['pid'].iloc[i]
print("Song provided:")
display(df_spotify.iloc[34][['track_name','artist_name','track_uri']])
x = generate_radio(test_uri, df_spotify_cosine, df_spotify, 10)
print()
print("Radio recommended")
display(x)


Song provided:


track_name                        Island In The Sun
artist_name                                  Weezer
track_uri      spotify:track:2MLHyLy5z5l5YRp7momlgw
Name: 34, dtype: object


Radio recommended


,track_name,artist_name,track_uri
1531,Sugar (feat. Francesco Yates),Robin Schulz,spotify:track:5tf1VVWniHgryyumXyJM7w
6525,Summer,Calvin Harris,spotify:track:6YUTL4dYpB9xZO5qExPf05
8596,Ni Una Sola Palabra,Paulina Rubio,spotify:track:52IreIJblsuK0SAJDIIh6v
7951,Yayo,Snootie Wild,spotify:track:1BfPC2cHY3m9RIstE4cjr5
4021,Closer,Ne-Yo,spotify:track:2nbClS09zsIAqNkshg6jnp
4546,Music Sounds Better,Big Time Rush,spotify:track:21svHgL8NlWthrlW9Gy0BZ
4171,Fading,William Bolton,spotify:track:2lnQRBzMUjSStGQRsGE8qU
8492,Take You There,Sean Kingston,spotify:track:2YUv5bLhf4ena4JWVY0Lql
8990,Valerie - '68 Version,Amy Winehouse,spotify:track:7Lxt392wmXBuWahE55fFAU


In [81]:
# This is the playlist generator, where you give a playlist and Spotify suggests you playlists based
# on that playlists

# This functions assume that the playlist you provide is already in the list of 10,000 songs we have for
# randomly selected 100 playlists

def generate_playlist(playlist, cosine_df, info_df, num_playlist = 2):
    '''
    Input:
    playlist = Playlist to provide recommendations
    cosine_df = Cosine similarity df
    info_df = The df with all the songs and track info and audio features
    num_playlist = Number of playlist to generate
    
    Output:
    rec_playlists = list of dfs of recommended playlist track names, artists and uri of recommended songs, length = num_playlists
    '''
    indices = list(info_df[info_df['track_uri'].isin(playlist)].index)
    #print("Playlist provided:")
    #display(info_df.iloc[indices][['track_name','artist_name','track_uri']])

    x = cosine_df.iloc[:,indices]

    my_list = []
    for i in range(len(indices)):
        my_list.append(list(x.iloc[:, i].sort_values(ascending = False)[1:num_playlist + 1].index))
    my_list = np.array(my_list).transpose()
    
    rec_playlist = []
    for i in my_list:
        rec_playlist.append(info_df.iloc[i][['track_name', 'artist_name', 'track_uri']])
    
    return rec_playlist

In [82]:
np.random.seed(seed = 10)

numbers3 = np.arange(0,len(df_spotify),1)
sample3 = np.random.choice(numbers3, size=10, replace=False)
test_playlist = df_spotify['track_uri']

rec_playlist = generate_playlist(test_playlist, df_spotify_cosine, df_spotify , 1)

counter = 0
for i in rec_playlist:
    counter += 1
    print("Recommended playlist #" + str(counter))
    display(i)
    print()

Recommended playlist #1


,track_name,artist_name,track_uri
6797,Saturday Night,Misfits,spotify:track:04EqqBVDCU2LOtxOMZ223T
7159,"Oh, Sister",Bob Dylan,spotify:track:4JtK4KieKw8mlPAIX4ODht
5017,Diamonds & Gold,Mac Miller,spotify:track:75bpA2kj6hAFeunmJ9IQie
7475,Poetic Justice,Kendrick Lamar,spotify:track:1zCi4cVFqe6ja16MeGZKRN
8330,Chelsea Dagger,The Fratellis,spotify:track:1bCmvezFg5MRcENzCGG1Cy
...,...,...,...
9372,Youngblood,Wage War,spotify:track:4t3lsnPp3ywXpvRxMWyJe1
114,Take Ü There (feat. Kiesza),Jack Ü,spotify:track:2RpKh7kXSdO8NLrW9VQ46p
9314,Revenant,Chelsea Grin,spotify:track:2jC2c2Ou7enoXPo1hhEzOV
1435,Aurora,Approaching Nirvana,spotify:track:7L3QsXEFNrhdxAPh7hFCkU


In [83]:
# This is one of the metrics that can be used to understand the precision of the recommendation
def r_precision(prediction, validation):
    score = np.sum(validation.isin(prediction))/validation.shape[0]
    return score

### NDCG Code Source: https://gist.github.com/bwhite/3726239
def dcg_at_k(r, k, method=0):
    r = np.asfarray(r)[:k]
    if r.size:
        if method == 0:
            return r[0] + np.sum(r[1:] / np.log2(np.arange(2, r.size + 1)))
        elif method == 1:
            return np.sum(r / np.log2(np.arange(2, r.size + 2)))
        else:
            raise ValueError('method must be 0 or 1.')
    return 0.


def ndcg_at_k(r, k, method=0):
    dcg_max = dcg_at_k(sorted(r, reverse=True), k, method)
    if not dcg_max:
        return 0.
    return dcg_at_k(r, k, method) / dcg_max

In [104]:
# Testing model

# Choosing pids with at least 100 songs such that our model gets at least 50 songs to work with
# The average playlist in our data set contains 52 songs
# So, if we take 100 songs at least and do a .5 train-test split
# The model will produce 50 recommendations and get tested on a 50 song playlist on average which makes sense

pids = list(df_spotify['pid'].value_counts()[df_spotify['pid'].value_counts() > 100].index)
r_precisions = []
ndcgs = []

for pid in pids:
    train, test = train_test_split(df_spotify[df_spotify['pid'] == pid], test_size=0.50, random_state = 24)
    prediction = rec_playlist = generate_playlist(train['track_uri'], df_spotify_cosine, df_spotify , 1)
    r = r_precision(prediction[0]['track_uri'], test['track_uri'])
    r_precisions.append(r)
    
    rx = np.zeros(len((prediction[0])))
    for i, p in enumerate(prediction[0]['track_uri']):
        if np.any(test['track_uri'].isin([p])):              
            rx[i] = 1
    ndcgs.append(ndcg_at_k(rx, len(rx)))

print("Average R precision: " + str(np.mean(r_precisions)))
print("Average NDCG: " + str(np.mean(ndcgs)))

Average R precision: 0.043582461509301736
Average NDCG: 0.2691182070784967
